<a href="https://colab.research.google.com/github/Dobby233Liu/colab-test-notebooks/blob/master/DeepDanbooru.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# DeepDanbooru (pretrained) image-evaluating notebook for Colab
This notebook will install and run DeepDanbooru. It not only lets you evaluate online without the Kanotype demo that's painfully slow and offline for a long time per a day, but also a way to try new models and/or codes.

---

# Setup

In [ ]:
# install package
!git clone --depth=1 https://github.com/KichangKim/DeepDanbooru.git
!pip install ./DeepDanbooru

In [ ]:
# download pretrained model
!mkdir model
!wget -O model/model.zip "https://github.com/KichangKim/DeepDanbooru/releases/download/v3-20200101-sgd-e30/deepdanbooru-v3-20200101-sgd-e30.zip" # change it in new releases
!cd model && unzip model.zip
!rm model/model.zip

In [9]:
import deepdanbooru as dd
import json

def _evaluate(image_path, project_path, threshold):
    model = dd.project.load_model_from_project(project_path, compile_model=False)
    tags = dd.project.load_tags_from_project(project_path)

    final_tags = []
    for tag, score in dd.commands.evaluate_image(image_path, model, tags, threshold):
            dObj = { "tag": tag, "score": score.item() }
            final_tags.append(dObj)

    return final_tags

---
# Evaluate image

In [ ]:
image_path = input("Image path (WARNING: this accepts only one file):\n")
print()
final_tags = _evaluate(image_path, "./model", 0.5)
print(f'Tag(s) for {image_path}\n')
for entry in final_tags:
      score = entry["score"]
      tag = entry["tag"]
      print(f'(~{score:05.3f}) {tag}')

# Evaluation server

In [ ]:
!wget https://bin.equinox.io/c/4VmDzA7iaHb/ngrok-stable-linux-amd64.zip -O ngrok-stable-linux-amd64.zip
!unzip -u ngrok-stable-linux-amd64.zip
!cp ngrok /bin/ngrok
!chmod +x /bin/ngrok
!/bin/ngrok authtoken 1ez5xDkBBHnburvW78pINIQ79WO_88HPQy5t9gULfh6UappJH

!pip install flask-ngrok
!pip install flask==0.12.2

In [ ]:
import flask
from flask_ngrok import run_with_ngrok
import werkzeug
import json
import os

app = flask.Flask(__name__)
run_with_ngrok(app)

@app.route('/predict', methods = ['POST'])
def handle_request_predict():
    imagefile = flask.request.files['image']
    filename = werkzeug.utils.secure_filename(imagefile.filename)
    imagefile.save(filename)

    final_tags = _evaluate(filename, "./model", 0.5)
    os.remove(filename)

    return json.dumps(final_tags)

if __name__ == '__main__':
    app.run()

Now you can:
```bash
curl -F "image=<jpg/png/gif>" http://<subdomain>.ngrok.io/predict
```
Example output:
```json
[{"tag": "1girl", "score": 0.9972099661827087}, {"tag": "animal_ears", "score": 0.9628053307533264}, {"tag": "bangs", "score": 0.6372907757759094}, {"tag": "beret", "score": 0.636321485042572}, {"tag": "black_headwear", "score": 0.5399508476257324}, {"tag": "blush", "score": 0.7675533890724182}, {"tag": "bow", "score": 0.5911067724227905}, {"tag": "bunny_ears", "score": 0.9958749413490295}, {"tag": "candy", "score": 0.8722140789031982}, {"tag": "collared_shirt", "score": 0.5831295847892761}, {"tag": "diagonal_stripes", "score": 0.9149599671363831}, {"tag": "eyebrows_visible_through_hair", "score": 0.5815466046333313}, {"tag": "food", "score": 0.9764244556427002}, {"tag": "hat", "score": 0.9660016894340515}, {"tag": "holding", "score": 0.8269818425178528}, {"tag": "holding_food", "score": 0.8598812818527222}, {"tag": "jacket", "score": 0.6863384246826172}, {"tag": "lollipop", "score": 0.7567343711853027}, {"tag": "long_hair", "score": 0.7823876142501831}, {"tag": "long_sleeves", "score": 0.5009376406669617}, {"tag": "looking_at_viewer", "score": 0.562648355960846}, {"tag": "necktie", "score": 0.6191784143447876}, {"tag": "shirt", "score": 0.846998929977417}, {"tag": "solo", "score": 0.9939253926277161}, {"tag": "striped", "score": 0.981475830078125}, {"tag": "striped_background", "score": 0.6125760674476624}, {"tag": "striped_neckwear", "score": 0.5707010626792908}, {"tag": "stuffed_animal", "score": 0.733761727809906}, {"tag": "stuffed_toy", "score": 0.6411434412002563}, {"tag": "twintails", "score": 0.9228045344352722}, {"tag": "upper_body", "score": 0.6335099339485168}, {"tag": "white_shirt", "score": 0.8326262831687927}, {"tag": "rating:safe", "score": 0.9996780157089233}]
```
You can also write a Web UI, but I'm not going to do that.